In [1]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/spark-2.4.7/spark-2.4.7-bin-hadoop2.7.tgz
!tar xf spark-2.4.7-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.7-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:14 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:15 http://ppa.launchpad.net/graph

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
# Start a SparkSession
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("word2vec").getOrCreate()

In [4]:
from pyspark.sql.types import StructType, StructField, StringType
from pyspark.ml.feature import Word2Vec, Word2VecModel
import string


In [5]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url ="/content/gdrive/MyDrive/Project3_BeerQuality/Resources/reviews_beer_brewery.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("reviews_beer_brewery.csv"), sep=",", header=True)

# Show DataFrame
df.show()

+-------+----------------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------+--------------------+--------------------+----+----------+--------------------+---------------+-------------+-----------------+
|beer_id|        username|      date|                text|                look|               smell|               taste|                feel|             overall|               score|           beer_name|review_state|               style|        availability| abv|brewery_id|        brewery_name|   brewery_city|brewery_state|    brewery_types|
+-------+----------------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------+--------------------+--------------------+----+----------+--------------------+-----------

In [6]:
def remove_punctuation(txt):
  txt = txt.strip('\xa0\xa0 ')
  return "".join(l if l not in string.punctuation else "" for l in txt)

def remove_space (text):
  if text:
    return text.replace(' ', '')
  else:
    return 'Unknown'
  

In [7]:
from pyspark.sql.types import StringType
from pyspark.sql.functions import col, udf

remove_punctuation_udf = udf(remove_punctuation, StringType())
remove_punctuation_udf

remove_space_udf = udf(remove_space, StringType())
remove_space_udf

<function __main__.remove_space>

In [8]:
beer_df = df.withColumn('beer_clean', remove_space_udf(col('beer_name')))
beer_df.show()

+-------+----------------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------+--------------------+--------------------+----+----------+--------------------+---------------+-------------+-----------------+--------------------+
|beer_id|        username|      date|                text|                look|               smell|               taste|                feel|             overall|               score|           beer_name|review_state|               style|        availability| abv|brewery_id|        brewery_name|   brewery_city|brewery_state|    brewery_types|          beer_clean|
+-------+----------------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------+--------------------+--------------------+----+-

In [9]:
style_df = beer_df.withColumn('style_clean', remove_space_udf(col('style')))
style_df.show()

+-------+----------------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------+--------------------+--------------------+----+----------+--------------------+---------------+-------------+-----------------+--------------------+--------------------+
|beer_id|        username|      date|                text|                look|               smell|               taste|                feel|             overall|               score|           beer_name|review_state|               style|        availability| abv|brewery_id|        brewery_name|   brewery_city|brewery_state|    brewery_types|          beer_clean|         style_clean|
+-------+----------------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------+------

In [10]:
text_df = style_df.withColumn("clean_text", remove_punctuation_udf(col("text")))
text_df.show()

+-------+----------------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------+--------------------+--------------------+----+----------+--------------------+---------------+-------------+-----------------+--------------------+--------------------+--------------------+
|beer_id|        username|      date|                text|                look|               smell|               taste|                feel|             overall|               score|           beer_name|review_state|               style|        availability| abv|brewery_id|        brewery_name|   brewery_city|brewery_state|    brewery_types|          beer_clean|         style_clean|          clean_text|
+-------+----------------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------------

In [11]:
#df.show(truncate=False)
from pyspark.sql.functions import concat_ws
combined_df = text_df.select(concat_ws(' ',text_df.beer_clean, text_df.style_clean, text_df.clean_text).alias("Final_text"))
#combined_df = text_df.select(concat_ws(' ',text_df.beer_clean, text_df.clean_text).alias("Final_text"))
#combined_df = text_df.select(concat_ws(' ',text_df.style_clean, text_df.clean_text).alias("Final_text"))
#combined_df = text_df.select('clean_text')
#combined_df = combined_df.withColumnRenamed('clean_text', 'Final_text')
combined_df.show(truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [12]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover
from pyspark.ml import Pipeline

# "Creating pipeline..."
tokenizer = Tokenizer(inputCol="Final_text", outputCol="token_text")
stopremove = StopWordsRemover(inputCol='token_text', outputCol='features')

pipeline = Pipeline(stages=[tokenizer, stopremove])

# "Training model..."
pipeline_stg = pipeline.fit(combined_df)
final_df = pipeline_stg.transform(combined_df)
# final_df.show()

In [13]:
final_df.show()

+--------------------+--------------------+--------------------+
|          Final_text|          token_text|            features|
+--------------------+--------------------+--------------------+
|MotorbreathImperi...|[motorbreathimper...|[motorbreathimper...|
|PaybackPilsner Bo...|[paybackpilsner, ...|[paybackpilsner, ...|
|PaybackPilsner Bo...|[paybackpilsner, ...|[paybackpilsner, ...|
|PaybackPilsner Bo...|[paybackpilsner, ...|[paybackpilsner, ...|
|PaybackPilsner Bo...|[paybackpilsner, ...|[paybackpilsner, ...|
|PaybackPilsner Bo...|[paybackpilsner, ...|[paybackpilsner, ...|
|itwasmuchbetteron...|[itwasmuchbettero...|[itwasmuchbettero...|
|PaybackPilsner Bo...|[paybackpilsner, ...|[paybackpilsner, ...|
|PaybackPilsner Bo...|[paybackpilsner, ...|[paybackpilsner, ...|
|PaybackPilsner Bo...|[paybackpilsner, ...|[paybackpilsner, ...|
|PaybackPilsner Bo...|[paybackpilsner, ...|[paybackpilsner, ...|
|HELLHAMMER RyeBee...|[hellhammer, ryeb...|[hellhammer, ryeb...|
|HELLHAMMER RyeBee...|[he

In [14]:
word2vec = Word2Vec(
    vectorSize=65,
    seed=42,
    inputCol="features",
    outputCol="model"
).setMaxIter(2)

In [ ]:
model = word2vec.fit(final_df)

In [ ]:
model.getVectors().show()

In [ ]:
model.findSynonymsArray("light", 5)

In [ ]:
model.write().overwrite().save("/content/gdrive/MyDrive/Project3_BeerQuality/review.md")

In [ ]:
loaded_model = Word2VecModel.load('/content/gdrive/MyDrive/Project3_BeerQuality/review.md')

In [ ]:
loaded_model.findSynonymsArray("corny", 5)